<a href="https://colab.research.google.com/github/donlap/stat424/blob/main/4_convex_program.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

In [ ]:
db = load_diabetes()

X = db.data
y = db.target

In [ ]:
print(X)

[[ 0.03807591  0.05068012  0.06169621 ... -0.00259226  0.01990749
  -0.01764613]
 [-0.00188202 -0.04464164 -0.05147406 ... -0.03949338 -0.06833155
  -0.09220405]
 [ 0.08529891  0.05068012  0.04445121 ... -0.00259226  0.00286131
  -0.02593034]
 ...
 [ 0.04170844  0.05068012 -0.01590626 ... -0.01107952 -0.04688253
   0.01549073]
 [-0.04547248 -0.04464164  0.03906215 ...  0.02655962  0.04452873
  -0.02593034]
 [-0.04547248 -0.04464164 -0.0730303  ... -0.03949338 -0.00422151
   0.00306441]]


In [ ]:
print(y)

[151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150. 279.
  92.  83. 128. 102. 302. 198.  95.  53. 134. 144. 232.  81. 104.  59.
 246. 297. 258. 229. 275. 281. 179. 200. 200. 173. 180.  84. 121. 161.
  99. 109. 115. 268. 274. 158. 107.  83. 103. 272.  85. 280. 336. 281.
 118. 317. 235.  60. 174. 259. 178. 128.  96. 126. 288.  88. 292.  71.
 197. 186.  25.  84.  96. 195.  53. 217. 172. 131. 214.  59.  70. 220.
 268. 152.  47.  74. 295. 101. 151. 127. 237. 225.  81. 151. 107.  64.
 138. 185. 265. 101. 137. 143. 141.  79. 292. 178.  91. 116.  86. 122.
  72. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_train

array([[-1.51413836, -0.93419873, -1.17946349, ..., -1.67619218,
        -1.86632264, -2.13184467],
       [ 0.5794076 ,  1.07043605,  1.24459041, ..., -0.01051158,
        -0.52326008, -0.35740392],
       [ 0.42433012, -0.93419873,  1.47327474, ..., -0.01051158,
        -0.40207853, -0.18840957],
       ...,
       [-1.74675458, -0.93419873, -1.56822685, ..., -0.84335188,
        -0.37859221, -1.70935877],
       [ 1.12217878,  1.07043605,  0.1926425 , ...,  0.82232873,
         1.32979127,  0.65656222],
       [ 1.19971752,  1.07043605,  1.10737982, ...,  0.82232873,
         1.20880708, -0.01941521]])

In [ ]:
X_test = scaler.transform(X_test)

In [ ]:
lasso = Lasso(alpha=20.)

lasso.fit(X_train, y_train)

lasso.coef_

array([ 0.        ,  0.        , 16.68723782,  2.82771722,  0.        ,
        0.        , -0.        ,  0.        , 13.85297711,  0.        ])

In [ ]:
!pip install cvxpy[SCIP]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 64.6 MB/s eta 0:00:00


In [ ]:
import cvxpy as cp

k = 4
M = 1000
d = X_train.shape[1]

beta = cp.Variable(d)
beta0 = cp.Variable()
z = cp.Variable(d, boolean=True)

In [ ]:
residual = y_train - (beta0 + X_train @ beta)
objective = cp.Minimize(cp.sum_squares(residual))
constraints = [cp.sum(z) <= k, beta <= M * z, beta >= - M * z]

problem = cp.Problem(objective, constraints)
problem.solve(cp.SCIP)

954381.6739859947

In [ ]:
beta.value

array([  0.        ,   0.        ,  27.34146492,  14.69183788,
       -12.10061407,   0.        ,   0.        ,   0.        ,
        30.42692754,   0.        ])

In [ ]:
beta0.value

array(152.08090918)

MSE = $\frac1{n_{\text{test}}}\sum_{i=1}^{n_{\text{test}}} (y_i - X_i^\top \hat\beta)^2$